### 회귀모델

In [2]:
# 회귀
# 출력의 개수를 1개로
# 손실함수를 MSE나 기타 등등..
# 데이터셋과 데이터 로드를 커스텀하게 정의해서 사용
# 나머지는 동일한 패턴으로... 학습 평가

In [3]:
#from sklearn.datasets import load_boston
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [4]:
# 데이터프레임 형식으로 데이터 가져옴
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

class BostonDataSet(Dataset):
    def __init__(self,X,y):

        self.X = torch.tensor(X, dtype=torch.float32)                # PyTorch가 계산할 수 있는 데이터 형태인 텐서(Tensor)로 변환
        self.y = torch.tensor(y, dtype=torch.float32).view(-1,1)     # view(-1, 1): y 텐서의 형태(shape)를 (데이터 개수, 1)인 2차원 배열로 변경
    def __len__(self):                      # 데이터셋의 총 데이터 개수를 반환하는 함수
        return len(self.X)
    def __getitem__(self, idx):             # 데이터셋[인덱스]와 같이 특정 위치의 데이터를 가져올 때 호출되는 함수
        return self.X[idx], self.y[idx]     # idx 번째의 문제지(X)와 그에 해당하는 정답지(y)를 한 쌍으로 묶어달라는 의미
                                            # 머신러닝 모델을 학습시킬 때는 항상 **'입력(X)'**과 그에 대한 **'정답(y)'**이 한 쌍으로 필요하기 때문에
                                            # 입력이 종속변수 x, 정답이 종속변수 y

배치단위로 데이터를 묶어준다

In [5]:
X_dataset = BostonDataSet(data, target)  # 데이터를 X, y 를 한쌍으로 묶어/ BostonDataSet 클래스의 실제 인스턴스(객체)를 만드는 과정
# data: 독립변수(X)들이 들어있는 NumPy 배열
# target: 종속변수(y)가 들어있는 NumPy 배열
# data와 target이 클래스로 전달되어, 클래스 내부의 self.X와 self.y에 각각 PyTorch 텐서 형태로 저장
# => : X_dataset이라는 변수에는 파이토치가 이해할 수 있는형식으로 포장
X_train_loader = DataLoader(X_dataset ,batch_size=32,shuffle=True)
# 만든 X_dataset을 모델을 훈련시킬 수 있는 데이터로 만드는 과정
# batch_size=32: 의미: 전체 데이터를 32개씩 작은 묶음(미니배치, mini-batch)으로 잘라서 사용하라는 의미


In [6]:
# 회귀모델 정의
class BostonRegression(nn.Module):  # 새로 생성하는 클래스 
    # PyTorch에서 모든 신경망 모델은 nn.Module이라는 기본 클래스를 상속받아야 함
    # 모델의 모든 학습 가능한 파라미터(가중치, 편향)를 추적하고 관리하는 기능)을 자동으로 갖게 됩
    def __init__(self, input_dim):  # input_dim -> 모델이 입력으로 받게 될 데이터의 특징(feature) 개수를 의미/ 나중에 보스턴 데이터셋의 개수인 13을 넣어준다
        super(BostonRegression,self).__init__()
        self.model = nn.Sequential(     # nn.Sequential: 여러 신경망 층을 순서대로 쌓아주는 컨테이너
            nn.Linear(input_dim,64),    # 첫 번쨰 선형층/ input_dim(13개)의 입력 특징을 받아 64개의 출력으로 변환
            nn.ReLU(),                  # 선형 층을 통과한 결과에 비선형성/ ReLU는 입력이 0보다 크면 그대로 두고, 0보다 작으면 0으로 만들어줌
            nn.Linear(64,32),           # 두 번째 선형층/ 64개의 특징을 입력으로 받아, 다시 32개의 새로운 특징으로 변환
            nn.ReLU(),
            nn.Linear(32,1),            # 마지막 출력층/  32개의 특징을 입력으로 받아 최종적으로 1개의 값을 출력
        )                               # 1개를 예측하는 이유는 선형회귀이기 떄문에
    def forward(self,x):                # 모델의 순전파
        return self.model(x)

In [7]:
from torch.optim import Adam
model = BostonRegression(data.shape[1]) # 어떤 모델 신경망을 쓸 것인가/ data.shape[1]는 (506,13)이라는 데이터 쉐입을 불러오는 것. 2번 즉 13을 불러오는 것
criterion = nn.MSELoss()                # 모델의 예측이 정답과 얼마나 다른지 어떻게 측정할 것인가/ nn.MSELoss: **평균 제곱 오차(Mean Squared Error)**를 계산하는 손실 함수
optim = Adam(model.parameters(), lr = 1e-3) # 옵티마이저 -> 덜 틀리는 방향으로 가중치를 어떻게 업데이트할 것인가
                                            # 옵티마이저는 틀린 정도를 바탕으로 모델의 가중치를 개선함
# MSELoss은 자동으로 미분한 값을 가지고 있는거고, 옵티마이저는 가지고 있는 값을 자동으로 업데이트
                                

In [8]:
# 1. 학습 환경 설정
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
epochs = 100
# 2. 학습루프
for epoch in range(epochs):
    tqdm_obj = tqdm(X_train_loader,desc=f'epoch : {epoch+1}/{epochs}')  # X_train_loader로부터 미니배치(여기서는 32개씩 묶인 데이터)를 하나씩 꺼내와서 학습을 진행
    loss_lists = 0
    for data, label in tqdm_obj:
        # 경사초기화
        optim.zero_grad()       # 이전 배치의 경사(gradient) 값이 남아있으면 새로운 계산에 영향을 주므로, 매번 새로운 배치를 학습하기 전에 경사 값을 0으로 깨끗하게 초기화
        # 순전파
        preds = model(data.to(device))  # 현재 배치의 입력 데이터(data)를 모델에 넣어 예측값(preds)을 계산
        # 손실계산
        loss = criterion(preds, label.to(device))   # 모델의 예측값(preds)과 실제 정답(label)을 손실 함수(criterion)에 넣어 '틀린 정도', 즉 손실(loss)을 계산
        loss_lists += loss.item()
        # 역전파
        loss.backward() # 계산된 손실을 바탕으로, 모델의 각 가중치가 손실에 얼마나 영향을 미쳤는지(기울기, gradient)를 역으로 계산
        # 가중치 업데이트
        optim.step()    # 옵티마이저(optim)가 loss.backward()에서 계산된 기울기(gradient)를 이용해 모델의 가중치를 '더 정답에 가까워지는 방향'으로 업데이트
        tqdm_obj.set_postfix({'loss' : f'{loss.item():.4f}'})
    avg_loss = loss_lists / len(X_train_loader)     # 한 에포크가 끝나면, 모든 배치의 손실을 합한 loss_lists를 전체 배치 개수로 나누어 그 에포크의 평균 손실을 계산
    print(f'epoch : {epoch+1} : avg loss : {avg_loss:.4f}')
        
# 3. 모델 저장
torch.save(model.state_dict(), 'bostonRegression.pth') 
# model.state_dict(): 모델의 구조는 제외하고, 학습을 통해 얻어진 가중치(weights)와 편향(biases) 값들만을 사전(dictionary) 형태로 추출
# torch.save(..., 'bostonRegression.pth'): 추출된 상태 사전을 bostonRegression.pth라는 파일 이름으로 저장

epoch : 1/100: 100%|██████████| 16/16 [00:00<00:00, 89.18it/s, loss=102.3991]


epoch : 1 : avg loss : 263.4054


epoch : 2/100: 100%|██████████| 16/16 [00:00<00:00, 133.00it/s, loss=96.5623]


epoch : 2 : avg loss : 106.5706


epoch : 3/100: 100%|██████████| 16/16 [00:00<00:00, 107.89it/s, loss=86.5768]


...

epoch : 98 : avg loss : 17.4479


epoch : 99/100: 100%|██████████| 16/16 [00:00<00:00, 162.75it/s, loss=10.1317]


epoch : 99 : avg loss : 17.0799


epoch : 100/100: 100%|██████████| 16/16 [00:00<00:00, 196.70it/s, loss=11.7337]


epoch : 100 : avg loss : 16.7576


In [15]:
from sklearn.metrics import r2_score

# 평가
# 1. 훈련된 모델 불러오기
model.load_state_dict(torch.load('bostonRegression.pth',map_location=device,weights_only=True))
# map_location=device: 모델을 불러올 장치를 지정
# weights_only=True: 보안을 위한 옵션으로, 파일에서 모델의 가중치(텐서)만 불러오도록 제한
# model.load_state_dict(...): 불러온 가중치 사전을 현재 model의 구조에 맞게 적용합니다. 이로써 model은 비어있는 초기 상태가 아니라, 100 에포크 동안 학습한 똑똑한 상태
# 예측
# 평가 루프
# 평가 모드 설정 및 준비        # 훈련 시에만 사용되던 Dropout이 비활성화되어 모델의 모든 뉴런이 예측에 사용됩
model.eval()  # 평가 모드로 전환 (dropout, batchnorm 등 비활성화)/ 모델의 일부 층(예: Dropout, BatchNorm)은 훈련할 때와 평가할 때 다르게 동작
total_mse = 0   # 전체 데이터셋에 대한 MSE(평균 제곱 오차)를 합산하기 위한 변수를 0으로 초기화

criterion = nn.MSELoss()
r2scores = 0
with torch.no_grad():  # 그래디언트 계산 비활성화/ 평가는 모델의 성능을 확인하는 단계일 뿐, 가중치를 업데이트(학습)하는 단계가 아니여서
    for data, label in tqdm(X_train_loader, desc="Evaluating"):
        data, label = data.to(device), label.to(device)
        preds = model(data)     # preds = model(data): 데이터를 모델에 넣어 예측값을 계산합니다. (순전파)
        r2scores += r2_score(label.cpu().detach().numpy(),preds.cpu().detach().numpy())
        mse = criterion(preds, label)   # mse = criterion(preds, label): 예측값과 실제 정답 사이의 MSE를 계산     
        total_mse += mse.item()
print(f"Test Loss: {total_mse / len(X_train_loader)} r2 score : {r2scores/len(X_train_loader)}")

Evaluating: 100%|██████████| 16/16 [00:00<00:00, 342.83it/s]

Test Loss: 16.300114452838898 r2 score : 0.7868169099092484
